In [67]:
import pandas as pd
import numpy as np
from PredictNan import PredictNan
import datetime
import math

In [68]:
train = pd.read_csv("../data/raw/d_train.csv")
test = pd.read_csv("../data/raw/d_test_A.csv")

test_id = test.pop("id")
train_id =  train.pop("id")

total_ID = pd.concat([train_id, test_id])

target = train.pop("血糖")


In [69]:
total = pd.concat([train,test])

In [70]:
columns = ["*r-谷氨酰基转换酶",
           "*球蛋白",
           "低密度脂蛋白胆固醇",
           "乙肝e抗体",
           "红细胞计数",
           "中性粒细胞%",
           ]
for column in columns:
    total[column] = total[column].apply(lambda x : np.log1p(x))


columns = [
          "红细胞平均血红蛋白量",
          "血小板比积"]
for column in columns:
    total.pop(column)
       

In [71]:
#添加统计量
#columns = total.columns.tolist()
columns = ["age","甘油三酯","红细胞平均体积","尿素","尿酸","*碱性磷酸酶","红细胞平均血红蛋白浓度","*天门冬氨酸氨基转换酶",
          "*r-谷氨酰基转换酶",
          ]
for column in columns:
    if column != "date" and column != "gender":
        max_value = total[column].max()
        min_value = total[column].min()
        avg_value = total[column].mean()
        total[column+"max"] = total[column].apply(lambda x : x - max_value)
        #total[column+"min"] = total[column].apply(lambda x : x - min_value)
        #total[column+"avg"] = total[column].apply(lambda x : x - avg_value)
'''
def normal(feature, value, height=1):
    """
    Determine whether the value is normal or missing
    :param feature:
    :param value: feature's value
    :param height: 1 means over the value is abnormal, 0 means under the value is abnormal
    :return: if normal return 0, abnormal return 1, missing return -1
    """
    if math.isnan(feature):
        return -1
    elif (height == 1 and feature > value) or (height == 0 and feature < value):
        return 1
    else:
        return 0
total['甘油三酯_normal'] = total['甘油三酯'].apply(normal, args=(1.69,))
total['尿素_normal'] = total['尿素'].apply(normal, args=(7.1,))
total['尿酸_normal'] = total.apply(lambda x: 0 if (x['gender']=='0' and x['尿酸']>357) or (x['gender']=='1' and x['尿酸']>416) else 1, axis=1)
total['*天门冬氨酸氨基转换酶_normal'] = total['*天门冬氨酸氨基转换酶'].apply(normal, args=(40,))
'''

'\ndef normal(feature, value, height=1):\n    """\n    Determine whether the value is normal or missing\n    :param feature:\n    :param value: feature\'s value\n    :param height: 1 means over the value is abnormal, 0 means under the value is abnormal\n    :return: if normal return 0, abnormal return 1, missing return -1\n    """\n    if math.isnan(feature):\n        return -1\n    elif (height == 1 and feature > value) or (height == 0 and feature < value):\n        return 1\n    else:\n        return 0\ntotal[\'甘油三酯_normal\'] = total[\'甘油三酯\'].apply(normal, args=(1.69,))\ntotal[\'尿素_normal\'] = total[\'尿素\'].apply(normal, args=(7.1,))\ntotal[\'尿酸_normal\'] = total.apply(lambda x: 0 if (x[\'gender\']==\'0\' and x[\'尿酸\']>357) or (x[\'gender\']==\'1\' and x[\'尿酸\']>416) else 1, axis=1)\ntotal[\'*天门冬氨酸氨基转换酶_normal\'] = total[\'*天门冬氨酸氨基转换酶\'].apply(normal, args=(40,))\n'

In [72]:
#删除日期
date = total.pop("date")
#total["date"]=total["date"].apply(lambda x : datetime.datetime.strptime(x, "%d/%m/%Y").isoweekday())


In [73]:
'''
predict = PredictNan(total, total_ID)
for column in predict.continue_columns:
    total[column] = predict.targets[column]
for column in predict.discrete_columns:
    total[column] = predict.targets[column]
'''

'\npredict = PredictNan(total, total_ID)\nfor column in predict.continue_columns:\n    total[column] = predict.targets[column]\nfor column in predict.discrete_columns:\n    total[column] = predict.targets[column]\n'

In [74]:
#离散化age
#total["age"] = total["age"].apply(lambda x : int(x/5.0+0.5))
#one-hot编码（gender和age）
one_hot = pd.get_dummies(total["gender"], prefix="gender")
total.pop("gender")
total = pd.concat([one_hot,total],axis=1)

#one_hot = pd.get_dummies(total["date"], prefix="date")
#total.pop("date")
#total = pd.concat([one_hot,total],axis=1)

In [75]:
total.head()

,gender_0,gender_1,age,*天门冬氨酸氨基转换酶,*丙氨酸氨基转换酶,*碱性磷酸酶,*r-谷氨酰基转换酶,*总蛋白,白蛋白,*球蛋白,...,嗜碱细胞%,agemax,甘油三酯max,红细胞平均体积max,尿素max,尿酸max,*碱性磷酸酶max,红细胞平均血红蛋白浓度max,*天门冬氨酸氨基转换酶max,*r-谷氨酰基转换酶max
0,0,1,41,24.96,23.10,99.59,3.055415,76.88,49.60,3.342155,...,0.6,-52,-40.26,-21.1,-7.52,-427.20,-274.73,-115.0,-409.99,-3.548515
1,0,1,41,24.57,36.25,67.21,4.382027,79.43,47.76,3.486457,...,0.8,-52,-38.76,-25.5,-8.13,-289.81,-307.11,-120.0,-410.38,-2.221904
2,0,1,46,20.82,15.23,63.69,3.667911,86.23,48.00,3.669442,...,0.8,-47,-40.58,-21.1,-8.62,-324.52,-310.63,-122.0,-414.13,-2.936019
3,1,0,22,14.99,10.59,74.08,3.054944,70.98,44.02,3.330775,...,0.5,-71,-40.51,-19.1,-9.16,-407.74,-300.24,-122.0,-419.96,-3.548986
4,1,0,48,20.07,14.78,75.79,3.166319,78.05,41.83,3.616846,...,0.6,-45,-40.60,-44.3,NaN,NaN,-298.53,-163.0,-414.88,-3.437612


In [76]:
#处理缺失值
columns = total.columns.tolist()
for column in columns:
    total[column] = total[column].fillna(total[column].mean())

In [77]:
#
train = total[0:5642]
train = pd.concat([train_id,train,target],axis=1)

test = total[5642:]
test = pd.concat([test_id,test],axis=1)

In [78]:
train.to_csv("../data/processed/train.csv",index=False,encoding="utf8")
test.to_csv("../data/processed/test.csv",index=False,encoding="utf8")